# Template Library Basics

This template intentionally ships only workshop-agnostic helpers in `simulated_city`:

- Load configuration from `config.yaml` + optional `.env` (`simulated_city.config`)
- Build MQTT topics and connect/publish (`simulated_city.mqtt`)

The actual city simulation is an exercise (see `docs/exercises.md`).

In [ ]:
# Imports used in this notebook.
import json

from simulated_city.config import load_config
from simulated_city.mqtt import publish_json_checked, topic

In [ ]:
# Safety switch: publishing sends a real MQTT message.
# Keep this False unless you really want to publish.
ENABLE_PUBLISH = False

In [ ]:
# Load settings from config.yaml (and .env for secrets, if present).
cfg = load_config()

In [ ]:
# Build an example topic + JSON payload (no network calls yet).
events_topic = topic(cfg.mqtt, "events/demo")
payload = json.dumps({"hello": "humtek"})

events_topic, payload

In [ ]:
# Print where the notebook would publish (helps debugging).
print("MQTT broker:", f"{cfg.mqtt.host}:{cfg.mqtt.port}", "tls=", cfg.mqtt.tls)
print("Base topic:", cfg.mqtt.base_topic)
print("Example publish topic:", events_topic)

## Optional: connect and publish

This section publishes **one** MQTT message so you can confirm your broker credentials work.

- Cell 3: set `ENABLE_PUBLISH = True`
- Cell 8: does the actual publish + prints what happened

Tip: in many broker dashboards you’ll want to subscribe to `simulated-city/#` (or your configured base topic + `/#`) to see messages.

In [ ]:
# If enabled, publish ONE message and (optionally) verify it by self-subscribing.
if not ENABLE_PUBLISH:
    print("Skipping MQTT publish (ENABLE_PUBLISH is False).")
    print("To enable: set ENABLE_PUBLISH = True in Cell 3.")
else:
    result = publish_json_checked(
        cfg.mqtt,
        topic=events_topic,
        payload=payload,
        client_id_suffix="notebook",
        wait_timeout_s=8.0,
    )
    print(result)
    if result.error:
        print("Issue:", result.error)

## CRS transforms (EPSG:3857 ↔ EPSG:25832)

Web maps commonly use **Web Mercator** (`EPSG:3857`).
For simulation math (meters), **ETRS89 / UTM zone 32N** (`EPSG:25832`) is often convenient.

In [ ]:
from simulated_city.geo import (
    epsg25832_to_webmercator,
    webmercator_to_epsg25832,
)

# Example point in Web Mercator meters (x, y)
x_3857, y_3857 = 1_000_000.0, 7_000_000.0

try:
    e_25832, n_25832 = webmercator_to_epsg25832(x_3857, y_3857)
    x_back, y_back = epsg25832_to_webmercator(e_25832, n_25832)

    {
        "3857": (x_3857, y_3857),
        "25832": (e_25832, n_25832),
        "back_to_3857": (x_back, y_back),
        "roundtrip_error_m": (x_back - x_3857, y_back - y_3857),
    }
except ModuleNotFoundError as e:
    print(str(e))
    print("Install with: pip install -e \".[geo]\"  (or: pip install pyproj)")

## Web map example (Copenhagen City Hall)

This section shows a point "in front of" Copenhagen City Hall on a web map.
We start with an approximate WGS84 lat/lon for the City Hall area, then compute 3857 + 25832 for display.

Note: Folium renders as a large HTML/JS output. VS Code can be slow/hang when rendering it inline.
Cell 12 only builds the map object `m`. Cell 13 lets you choose: inline display (JupyterLab) or save HTML (VS Code).

Folium is optional; if it’s not installed, the cell prints install instructions.

In [ ]:
# Approximate point near Copenhagen City Hall (Rådhuspladsen).
# If you need an exact surveyed point later, replace these numbers.
lat, lon = 55.6761, 12.5683

try:
    from pyproj import Transformer  # type: ignore
    from simulated_city.geo import EPSG_3857, webmercator_to_epsg25832
except ModuleNotFoundError as e:
    print(str(e))
    print("Install with: pip install -e \".[geo]\"  (or: pip install pyproj)")
else:
    # WGS84 (EPSG:4326) -> Web Mercator (EPSG:3857)
    to_3857 = Transformer.from_crs("EPSG:4326", EPSG_3857, always_xy=True)
    x_wm, y_wm = to_3857.transform(lon, lat)

    # Web Mercator (3857) -> UTM32 (25832) for meter-based math
    e_utm, n_utm = webmercator_to_epsg25832(x_wm, y_wm)

    print("WGS84 (lat, lon):", (lat, lon))
    print("EPSG:3857 (x, y):", (x_wm, y_wm))
    print("EPSG:25832 (E, N):", (e_utm, n_utm))

    # Build a simple web map (folium)
    try:
        import folium  # type: ignore
    except ModuleNotFoundError:
        folium = None

    if folium is None:
        print("\nfolium is not installed.")
        print("Install with: pip install -e \".[notebooks]\"  (or: pip install folium)")
    else:
        m = folium.Map(location=[lat, lon], zoom_start=18, tiles="OpenStreetMap")
        popup = folium.Popup(f"EPSG:25832 (m): E={e_utm:.2f}, N={n_utm:.2f}", max_width=300)
        folium.Marker([lat, lon], popup=popup, tooltip="Copenhagen City Hall (approx)").add_to(m)

In [ ]:
from pathlib import Path

# Choose how to view the map:
# - JupyterLab: set INLINE_FOLIUM=True for inline rendering
# - VS Code: keep INLINE_FOLIUM=False and open the saved HTML file
INLINE_FOLIUM = True

if "m" not in globals():
    print("No map object `m` was created (folium may be missing).")
elif INLINE_FOLIUM:
    # Works even if this cell has other output.
    from IPython.display import display  # type: ignore
    display(m)
else:
    map_html_path = Path("copenhagen_city_hall_map.html").resolve()
    m.save(str(map_html_path))
    print("Saved map HTML:", map_html_path)
    print("Open it in your browser (file URL):", map_html_path.as_uri())

In [ ]:
# Intentionally left blank.
# (Avoid rendering the Folium map inline by accident.)